**Attribution:**  

This notebook was modified from Debsankha Manik's notebook Pandas, GGNB Data Science course held at the University of Goettingen (2019).


In [ ]:
import pandas as pd
import numpy as np
from urllib import request


# [Pandas](https://pandas.pydata.org)

* Read and manipulate tabular data
* Based on NumPy arrays
* Unlike NumPy arrays, Pandas dataframes can handle different data types

<a title="Michael Droettboom [BSD (http://opensource.org/licenses/bsd-license.php)], via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:Pandas_logo.svg"><img width="350" alt="Pandas logo" src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/ed/Pandas_logo.svg/512px-Pandas_logo.svg.png"></a>  


## Data I/O

* csv
* json
* hdf
* html
* many more

Now, let's assume we have already gotten our dataset loaded into pandas.
Next job is to do meaningful analysis on the data.

### Inspecting data

In [ ]:
birds_filepath = '../data/amazonian_birds.csv'
df0 = pd.read_csv(birds_filepath, parse_dates={'datetime':[1,2]}, error_bad_lines=False)
print(df0.shape, df0.columns)
df0.head()

In [ ]:
df0.sample(n = 4)

In [ ]:
df0.dtypes

In [ ]:
df0.mean()

## Sorting data by date

In [ ]:
df = df0.sort_values('datetime')
df.head()

## Selection

`df.iloc[:]` selecting by row number [doc](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#indexing-integer)

`df.loc[:]` selecting by label [doc](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#indexing-label)

### Getting rows by row number

In [ ]:
df0.iloc[:14:2]

In [ ]:
df0.loc[:14:2]

In the above example `.loc` and `.iloc` produce the same outcome because the index matches the row positions, but this is not always the case as we will see below.

The syntax `df[0:4]` also works, but can yield unexpected outcomes, see caveats below. 


## Filtering out  with Boolean indexing
*we saw it already in the morning*

In [ ]:
df = df0[df0['datetime'] >= '2000-01-01']

In [ ]:
df.tail()

In [ ]:
%%latex
\[
\texttt{df = df[}\underbrace{\texttt{df['datetime']>='1970-01-01'}}_{\texttt{Boolean array}}]
\]

### Quite complex filetering is also possible

In [ ]:
df0[(df0['datetime'] > '2012-07-01') & (df0['datetime'] < '2012-08-01')].head()

## Transforming data types

In [ ]:
df0.dtypes

In [ ]:
df0['elevation'].astype(float)

In [ ]:
def coerce_float(x):
    '''try to convert to float otherwise fill with nan'''
    try:
        return float(x)
    except ValueError: 
        return None

In [ ]:
df0['elevation'] = df0['elevation'].apply(coerce_float)

## Reindex by datetime

In [ ]:
df = df0.set_index('datetime')

In [ ]:
df.head()

## Selecting by **index label**

In [ ]:
df.loc['1990-07-01':'1992-12-01']

This is equivalent to

In [ ]:
df['1990-07-01':'1992-12-01']

**WARNING:** The `.loc` syntax is infinitely preferable for selecting by label, as then you avoid pitfalls like:

In [ ]:
td = pd.DataFrame(np.random.randint(10, size = (8,4)), 
                  index = range(3,11), 
                  columns=['A', 'B', 'C', 'D'])

In [ ]:
td

In [ ]:
td[0:4] #== td.iloc[0:4]

This does not return rows with index label between 0 and 4, however `.loc` does

In [ ]:
td.loc[0:4]

Why: the slicing operator `[:]` tries `iloc` first, then falls back to `loc`

### Slicing with increments

In [ ]:
td

In [ ]:
td.iloc[:5:2]

In [ ]:
td.iloc[::-1]

In [ ]:
td

In [ ]:
td.loc[4:6]

**WARNING:** Unlike Python's array indexing, `df.loc` *includes the endpoints* of the slices

In [ ]:
td

Get elements like with fancy indexing

In [ ]:
td.iloc[[3, 4, 5, 3]]

get only one element

In [ ]:
td.iloc[2,0]

## Benefits of indexing

In [ ]:
df = pd.read_csv(birds_filepath, parse_dates={'datetime':[1,2]}, error_bad_lines=False)
df.head()

In [ ]:
%timeit df[(df['datetime'] > '2011-02-24') & (df['datetime'] < '2013-04-28') ].count()

In [ ]:
idf = df.copy()
idf['datetime'] = pd.to_datetime(idf['datetime'], errors='coerce')
idf = idf[pd.notnull(df.datetime)]
idf = idf.set_index('datetime')
idf.head()

In [ ]:
%timeit idf['2011-02-24':'2013-04-28']['latitude'].count()

## Selecting by columns

In [ ]:
df.loc[:, ['recordist', 'latitude', 'longitude']].head()

# Transforming data

## Adding a column for the year

In [ ]:
df = pd.read_csv(birds_filepath, parse_dates={'datetime':[1,2]}, error_bad_lines=False)
df = df[pd.notnull(df.datetime)]
df = df.set_index('datetime')
df.head()

In [ ]:
df.loc[:, 'year'] = df.index.year
df.head()

## Applying transformations on a column

Add a column for year starting on the first day of recording.

In [ ]:
first_year = df.loc[:, 'year'].min()
print(first_year)

In [ ]:
df.loc[:, 'year'] = df.loc[:, 'year'] - first_year
df.head()

In [ ]:
df.loc[df['year'] < 3]

## Append

In [ ]:
df1 = pd.DataFrame(np.random.randint(5, size=(4,6)), columns=list('ABCDEF'))
df2 = pd.DataFrame(np.random.randint(5, size=(4,6)), columns=list('ABCDEF'))

In [ ]:
df1

In [ ]:
df2

In [ ]:
df3 = df1.append(df2)
df3

Now df3 has nonsensical index:

In [ ]:
df3.loc[2:3]

We have to reindex:

In [ ]:
df3

In [ ]:
df3.index = range(len(df3))

In [ ]:
df3

## Concat, append and merge


### Concat

<a href="https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html"><img width="500" src="https://pandas.pydata.org/pandas-docs/stable/_images/merging_concat_mixed_ndim.png"></a> 

### Append

<a href="https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html"><img width="500" src="https://pandas.pydata.org/pandas-docs/stable/_images/merging_concat_ignore_index.png"></a> 


### Merge

<a href="https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html"><img width="500" src="https://pandas.pydata.org/pandas-docs/stable/_images/merging_merge_on_key.png"></a>  

